## Introduction

There a many ways how to approach programming a quantum computer. The two extreme cases are
* manual development of an algorithm for each problem. This is similar to how code for HPC is developed, highly customized and highly specialized for each problem with high effort
* use a `high level compiler` to transform an abstract high level description of the problem to a machine executable form. The great advantage is that users don't need any knowledge about the underlying solver or its (quantum) implementation.

## Prototype

As a `prototype` for such a high level compiler we use a subset of Mathematical Programming called QUBO (Quadratic Unconstrained Binary Optimization) that is provided since Aqua 0.7.0 as `QuadraticProgram`.

* DONE: use subset of Mathematical Programming problems as stand-in for high level language
* DONE: automatically map QUBO to Eigenvector based Quantum Solvers
* DONE: use cloud interface to IBM Q to get the Quantum Circuits executed
* TODO: Provide complete sequence from CPLEX input to solution using IBM Q as a cloud service aka "Q backend for CPLEX"

## Example

Here we use bin packing as an example problem. It's intentionally kept small and simple to concentrate on the workflow below.

Just insert any other more complex model here and reuse the transformation and solver steps below.

In [1]:
from docplex.mp.model import Model

# Create an instance of a model and variables
mdl = Model(name='bin_pack')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(1,4)}

# Objective function
max_vars_func = mdl.sum(x[i] for i in range(1,4))
mdl.maximize(max_vars_func)

# Constraints
weight = [2, 4, 6]
mdl.add_constraint(mdl.sum(weight[i-1]*x[i] for i in range(1,4)) == 10)

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: x_1 + x_2 + x_3
Subject To
 c1: 2 x_1 + 4 x_2 + 6 x_3 = 10

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



## Transpile to QUBO

All conditions are moved to the objective term via qiskit converters.

In [4]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing, InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

# define operators and converters
qp = QuadraticProgram()
ineq2eq = InequalityToEquality()
lineq2penalty = LinearEqualityToPenalty()

# import DOCplex model and apply converters
qp.from_docplex(mdl)
qp_eq = ineq2eq.convert(qp)
qubo = lineq2penalty.convert(qp_eq)

print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: 161 x_1 + 321 x_2 + 481 x_3 + [ - 32 x_1^2 - 128 x_1*x_2 - 192 x_1*x_3
      - 128 x_2^2 - 384 x_2*x_3 - 288 x_3^2 ]/2 -400
Subject To

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



## Solvers (aka Compilers)

Now that we finally prepared the Quadratic Program for the problem above, we have several options to solve it.

Solving the QUBO involves transforming it into one or more Quantum Circuits which is equivalent to a compilation from the high level QUBO description down to a Quantum Gate description. The circuit(s) are then executed and the measured results interpreted and transformed back to the problem domain.

* Future advanced solvers might make use of `problem domain encoding` and need to implement this back transformation.
* The existing Eigenvector based solvers can skip this step because the minimum Eigenvector directly encodes the solution.
* For the Grover family it can be skipped as well because the best sample in the shifted prepared quantum state also encodes the solution directly.

The following examples will show how the Eigenvector based and Grover search based solvers are applied.

In [11]:
# Example: Eigenvector based solvers
from qiskit import BasicAer
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA, VQE, ExactEigensolver, NumPyMinimumEigensolver

# prepare Eigenvector based solvers
vqe_mes = VQE(quantum_instance=BasicAer.get_backend('statevector_simulator'))
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
vqe = MinimumEigenOptimizer(vqe_mes)   # using VQE
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

# solve and print
exact_result = exact.solve(qubo)
print("exact:", exact_result)
qaoa_result = qaoa.solve(qubo)
print("qaoa: ", qaoa_result)
#vqe_result = vqe.solve(qubo)
#print("vqe:  ",vqe_result)

exact: optimal function value: 2.0
optimal value: [0. 1. 1.]
status: SUCCESS
qaoa:  optimal function value: 2.0
optimal value: [0. 1. 1.]
status: SUCCESS


In [12]:
# Example: GAS `GroverOptimizer`
from qiskit.optimization.algorithms import GroverOptimizer

# prepare GAS solver
gas_mes = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=gas_mes)

# solve and print
gas_result = grover_optimizer.solve(qubo)
print("gas:  ", gas_result)

gas:   optimal function value: -62.0
optimal value: [1 1 0]
status: SUCCESS


## QPLEX - Quantum Acceleration for CPLEX

### Context
* Quantum Computers are Accelerators for highly parallel problems
* ML (QSVM, Perceptron) and DL are sexy, but require huge Quantum Computers with low error rate
* Near term: Solving Optimization Problems on real quantum hardware

### Dreamland
* quadratically better runtime (enables much larger problems than what is feasible today)
* higher quality solutions (aka big savings) due to global optimization
* cloud service on QC (Utz)
* dispatcher that selects between QC and Classical Solvers

### Obstacles
* NISQ: small -> issue with width
* NISQ: noisy -> issue with depth
* handle continuous variables in optimization problems efficiently

### Solution
* member of Quantum Group in BOE Lab since 2017. Patent on Quantum Safe Authentication. Invented 6qubit QECC.
* solve NISQ width problem -> RASO:	Relaxed Adaptive Subproblem Optimization (TB, Stefan Woerner)
* solve NISQ noise problem -> QECC (TB) OR shallow circuits (IBM research) OR surface codes (IBM research)
* solve data type problem -> approximate with many qubits (done) or hybrid classical/qubit registers (TB)

### Next steps
* Prototype End2End: Rest interface for CPLEX cloud offering integration (Daniel Junglas)
* Time: How to organize time for a systems guy, in the firmware org that works for a Data&AI product :-D 
* Help: REST interface skill, backing from Namik for a QPLEX offering


___________________________________________







# BACKUP








____________________________________________

## Improvements

### convenience: 

The usability and convenience has been improved lately by supporting models with
* linear and quadratic equations
* equalities and inequalities
* binary and integer variables

Most of these improvements of scope have been paid for in the number of qubits (slack variables, int approximation).
This makes the representation less compact and with that less fit for near term Quantum Computers.


### depth:

Recursive solvers are introduced to limit the Quantum Circuit depth.
The circuit is iteratively refined and executed many times.
Qiskit provides this via `RecursiveMinimumEigenOptimizer`.

### scope: 

* Binary optimization (QUBO): On top of Ising based solvers, Grover based solvers (`GroverOptimizer`) have been recently introduced.
* Mixed-binary optimization (MBCO): The ADMM Optimizer can solve classes of mixed-binary constrained optimization problems
* Mixed-integer methods (MILP) are still missing.

### width:

Future research is needed on hybrid solvers that process hard and compact subproblems on the Quantum Computer and combine this via classical solvers to the final solution.

### data representation:

Conversion between different representations for various algorithms. Library support needed.


In [8]:
# Example of the usage of the recursive solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

optimal function value: -14.0
optimal value: [1. 0. 1.]
status: SUCCESS


In [9]:
from qiskit import IBMQ
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers



C:\Users\TobiasBergmann\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [10]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright